# Predict and Label: 8 Way

## Overview

*   This notebook is to get a labeled training set for the deep neural network that I will train to pick the best model.





### Step 1: Clone the Repo

In [ ]:
#This will clone the BERT Repo

!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 317.20 KiB | 4.12 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [ ]:
#The code in the BERT Repo is written in tf 1, and the tf conversion process fails on these files.
#For this reason, it was easiest to revert to tf v1 for the purposes of this notebook

%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
#Make sure were in the right place

%ls

bert/  sample_data/


In [ ]:
# Move to BERT folder 

%cd bert

/content/bert


### Step 2: Imports and Connect to TPU

In [ ]:
'''I think I can just use a GPU for this and maybe TF 2.0
I wont need to move the model to the cloud.'''

# Still need imports 

import datetime
import json
import os
import time
import pprint
import random
import string
import sys
import tensorflow as tf
import re
from collections import Counter
from itertools import groupby
import pandas as pd
import numpy as np

# Get TPU Address for training

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is => ', TPU_ADDRESS)

#Authorize Google and connect.

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())
  
  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
    
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)



TPU address is =>  grpc://10.2.239.130:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 12592937843177746946),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4777036311942658929),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 5780418987813282082),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 3231926732534625904),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 160501029922996852),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 13948876096021055364),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 191708433020692316),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 2282562054462189422),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 259207603684

In [ ]:
#Create variables for Buckets and Outputs for later use. 

BUCKET = 'thaddeussegura_final_project' #@param {type:"string"}
assert BUCKET, '*** Must specify an existing GCS bucket name ***'
output_dir_name = 'self_ensemble_8/' #@param {type:"string"}
BUCKET_NAME = 'gs://{}'.format(BUCKET)
OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET,output_dir_name)
tf.io.gfile.makedirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

***** Model output directory: gs://thaddeussegura_final_project/self_ensemble_8/ *****


### Step 3: Connect Drive and GCP to get train/pred data


In [ ]:
#Mount my drive so that I can access the split training sets. 

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#Copy the data from drive to colab. 

#I am not sure that I need any files from my drive?
#Maybe the specific training sets?


'/content/bert'

In [ ]:
# Download the SQUAD train and dev dataset

# I will need the full training set.  
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

# Still download the Dev set.
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json

--2020-07-12 19:47:06--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  59.5MB/s    in 0.7s    

2020-07-12 19:47:07 (59.5 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]

--2020-07-12 19:47:08--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/json]
Saving to: ‘dev-v2.0.json’

dev-v2.0.json       

In [ ]:
# Necessary installs so I can mount the files from my bucket onto colab

!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  31095      0 --:--:-- --:--:-- --:--:-- 31095
OK
81 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 81 not upgraded.
Need to get 4,278 kB of archives.
After this operation, 12.8 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 144379 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.30.0_amd64.deb ...
Unpacking gcsfuse (0.30.0) ...
Setting up gcsfuse (0.30.0) ...


In [ ]:
# Make a folder for the bucket, this will have all of the files inside. 

!mkdir folderOnColab
!gcsfuse thaddeussegura_final_project folderOnColab 

Using mount point: /content/bert/folderOnColab
Opening GCS connection...
Opening bucket...
Mounting file system...
File system has been successfully mounted.


### Step 4: Get Model Accuracy

*   Take Each Model and predict it on the full dataset
*   Compare the pred answers to true answers
*  Output to build the dataset for the DNN training



In [ ]:
#need to get the checkpoint names as they differ in each folder. 

def find_ckpt(folder_name):
  path = 'folderOnColab/self_ensemble_8/'+folder_name
  file_list = os.listdir(path)
  max = 0
  for x in file_list:
    if 'model.ckpt-' in x:
      split = re.split('-|.meta|.index|.data', x)
      #print(int(split[1]))
      if int(split[1]) > max:
        max = int(split[1])
  ckpt_name = OUTPUT_DIR+folder_name+'/model.ckpt-'+str(max)
  return ckpt_name

test = find_ckpt('8_way_0')   
test

'gs://thaddeussegura_final_project/self_ensemble_8/8_way_0/model.ckpt-2116'

In [ ]:
# Prediction function

# This will train in a loop, but will also likely crash the instance. 

def full_train_pred(checkpoint, temp_output):
  !python run_squad.py \
    --vocab_file=$BUCKET_NAME/cased_L-24_H-1024_A-16/vocab.txt \
    --bert_config_file=$BUCKET_NAME/cased_L-24_H-1024_A-16/bert_config.json \
    --init_checkpoint=gs:$checkpoint \
    --do_train=False \
    --max_query_length=30  \
    --do_predict=True \
    --predict_file=train-v2.0.json \
    --predict_batch_size=8 \
    --n_best_size=3 \
    --use_tpu=True \
    --tpu_name=grpc://10.60.237.130:8470 \
    --max_seq_length=384 \
    --doc_stride=128 \
    --output_dir=$temp_output

full_train_pred('gs://thaddeussegura_final_project/self_ensemble_8/8_way_7/model.ckpt-2094', 'gs://thaddeussegura_final_project/self_ensemble_8/8_way_7/')


Streaming output truncated to the last 5000 lines.
I0711 23:37:36.164325 139915655210880 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0711 23:37:36.181219 139915655210880 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0711 23:37:36.181361 139915655210880 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0711 23:37:36.198577 139915655210880 tpu_estimator.py:600] Enqueue next (1) batch(es) of data to infeed.
INFO:tensorflow:Dequeue next (1) batch(es) of data from outfeed.
I0711 23:37:36.198745 139915655210880 tpu_estimator.py:604] Dequeue next (1) batch(es) of data from outfeed.
INFO:tensorflow:Enqueue next (1) batch(es) of data to infeed.
I0711 23:37:36.215595 139915655210880 tpu_estimator.py:600] Enqueue next (1) batch(es) of data

In [ ]:
# run the loop.
# Tends to crash the instance....


def full_pred_loop(splits):
  start_time = time.time()
  for i in range(0,splits):
    folder_name = '8_way_'+str(i)
    checkpoint = find_ckpt(folder_name)
    temp_output = OUTPUT_DIR+folder_name+'/'
    full_train_pred(checkpoint, temp_output)
    time.sleep(30) #Give the RAM a chance to reset.
  end_time = time.time()
  return end_time-start_time


### Step 5: Build the training labels 

*   1 for exact match
*   0 is its wrong
*   Return a master dataset with an array for each question and the outcomes of the prediction for each model.
* EX: [1,1,0,1,0,1]

In [ ]:
# This will take split data and return the keys that are present.
def extract_split_keys(data):
    keys = []
    for i in range(len(data['data'])):
        paragraphs = len(data['data'][i]['paragraphs'])
        for j in range(paragraphs):
            qas = len(data['data'][i]['paragraphs'][j]['qas'])
            for k in range(qas):
                keys.append(data['data'][i]['paragraphs'][j]['qas'][k]['id'])
    return keys

#this will be used to get the full answer list. 
def extract_answer_text(data):
    text = []
    for i in range(len(data['data'])):
        paragraphs = len(data['data'][i]['paragraphs'])
        for j in range(paragraphs):
            qas = len(data['data'][i]['paragraphs'][j]['qas'])
            for k in range(qas):
                if data['data'][i]['paragraphs'][j]['qas'][k]['is_impossible'] == True:
                    text.append("")
                else:
                    text.append(data['data'][i]['paragraphs'][j]['qas'][k]['answers'][0]['text'])
    return text

# Need a helper function to open each file
def open_json(path):
    with open(path) as json_file:
        temp_json = json.load(json_file)
        return temp_json

#pass in any set of full predictions to find the full keys. 
def extract_keys(data):
    predictions = []
    for group in data:
        predictions.append(group)
    return predictions

def code_preds(full_keys,temp_keys,full_answers,temp_preds):
  values = []
  for i,key in enumerate(full_keys):
      if key in temp_keys:
          values.append(0)
      else:
          if temp_preds[key] == full_answers[i]:
              values.append(1)
          else:
              values.append(-1)
  return values

def code_preds_no_zero(full_keys,temp_keys,full_answers,temp_preds):
  values = []
  for i,key in enumerate(full_keys):
    if temp_preds[key] == full_answers[i]:
        values.append(1)
    else:
        values.append(-1)
  return values

In [ ]:
#This will iterate through each of the predictions on the full
#Dataset made by each model and see if its prediction was right or wrong.
#It will add all of these to a dataframe which is then written to CSV
#After this I can pick up where I left off.

def build_csv(splits):
  full_train_data = open_json('train-v2.0.json')
  full_answers = extract_answer_text(full_train_data)
  for i in range(splits):
    path = 'folderOnColab/self_ensemble_8/8_way_'+str(i)+'/predictions.json'
    temp_preds = open_json(path)
    full_keys = extract_keys(temp_preds)
    split_path = '/content/drive/My Drive/8_way_split/8_way_'+str(i)+'.json'
    split_data = open_json(split_path)
    split_keys = extract_split_keys(split_data)
    values = code_preds_no_zero(full_keys,split_keys,full_answers,temp_preds)
    model_name = 'model_'+str(i)
    if i == 0:
      answer_df = pd.DataFrame(full_keys, columns = ['keys'])
      answer_df[model_name] = values
    else:
      answer_df[model_name] = values
  answer_df.to_csv('coded_preds.csv')

build_csv(8)


In [ ]:
# Reopen the CSV and cast back to DF.
# I can pick up here.

answers_df = pd.read_csv('folderOnColab/self_ensemble_8/coded_preds.csv')
answers_df = answers_df.drop(['Unnamed: 0','keys'], axis = 1)
answer_array = answers_df.to_numpy()
answer_array.shape

array([-1, -1, -1, -1, -1, -1, -1, -1])